<a href="https://colab.research.google.com/github/Rahulraj31/Fake-News-LSTM/blob/main/Bi_directional_LSTM_Fake_News_Classifieripynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Fake News Classifier Using Bi- directional LSTM


Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/fake-news.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/fake-news.zip'

In [5]:
import pandas as pd
df= pd.read_csv("train.csv")

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


**1: unreliable**<br>

**0: reliable**

In [7]:
df.shape

(20800, 5)

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df=df.dropna()

In [10]:
# Get the Independent and Dependent Features
X=df.drop('label',axis=1)
Y=df['label']

In [11]:
X.shape,Y.shape

((18285, 4), (18285,))

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences  #for proper same size input to embedding layer, same as widow in genshim
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense,Dropout

In [13]:
# vocabulary size
voc_size =5000

## One hot Representation


In [14]:
messages=X.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\weoni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages["title"][i]) #substitute everthing with blank apart from letters a-z and A-Z in title
  
  review =review.lower() #convert to lower case
  
  review=review.split() 

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #stems words no present in stopword
  review = ' '.join(review) #combining list of words to a list of sentences
  corpus.append(review)
  

In [19]:
onehot_rep= [one_hot(words,voc_size) for words in corpus]

In [20]:
# one hot for first 5 sentences
onehot_rep[0:5]

[[3000, 4439, 4795, 3802, 3810, 2170, 3878, 2559, 3979, 3824],
 [710, 4947, 4505, 4195, 2506, 3815, 1175],
 [3179, 2349, 4666, 636],
 [3991, 4816, 1594, 2138, 3604, 1979],
 [2741, 2506, 1623, 879, 4273, 2470, 2506, 26, 1662, 1478]]

## Embedding Representation

In [21]:
sent_length =20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2559 3979 3824]
 [   0    0    0 ... 2506 3815 1175]
 [   0    0    0 ... 2349 4666  636]
 ...
 [   0    0    0 ... 4267 2448 3480]
 [   0    0    0 ... 3986  166 4744]
 [   0    0    0 ... 2645 2975 3699]]


In [22]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  710, 4947, 4505, 4195, 2506, 3815, 1175])

In [23]:
# creating model
embedding_vector_features =40   #dimwnsion of feature vector of embedding 
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100)))  # 1 layer 100 neuron
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))


In [24]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
__________________________________________________

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [28]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=32)


Epoch 1/10
383/383 [==============================] - 29s 52ms/step - loss: 0.2755 - accuracy: 0.8722 - val_loss: 0.1980 - val_accuracy: 0.9157
Epoch 2/10
383/383 [==============================] - 20s 53ms/step - loss: 0.1464 - accuracy: 0.9418 - val_loss: 0.1933 - val_accuracy: 0.9191
Epoch 3/10
383/383 [==============================] - 21s 55ms/step - loss: 0.1069 - accuracy: 0.9602 - val_loss: 0.2094 - val_accuracy: 0.9163
Epoch 4/10
383/383 [==============================] - 25s 64ms/step - loss: 0.0880 - accuracy: 0.9700 - val_loss: 0.2333 - val_accuracy: 0.9160
Epoch 5/10
383/383 [==============================] - 22s 58ms/step - loss: 0.0656 - accuracy: 0.9773 - val_loss: 0.2878 - val_accuracy: 0.9137
Epoch 6/10
383/383 [==============================] - 24s 62ms/step - loss: 0.0542 - accuracy: 0.9807 - val_loss: 0.3021 - val_accuracy: 0.9153
Epoch 7/10
383/383 [==============================] - 27s 71ms/step - loss: 0.0494 - accuracy: 0.9825 - val_loss: 0.3036 - val_accuracy:


## Performance Metrics And Accuracy

In [32]:
y_pred=model.predict_classes(X_val)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_val,y_pred)

NameError: name 'y_pred' is not defined

In [36]:
accuracy_score(y_val,y_pred)

0.9154929577464789

In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3419
           1       0.88      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.92      0.92      6035

